In [ ]:
# Imports
import numpy as np
import os
from pathlib import Path

from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, RigidTransform,
    FindResourceOrThrow, RollPitchYaw, InverseDynamicsController,
    MultibodyPlant, Parser, Simulator, Role)

from pydrake.systems.meshcat_visualizer import MeshcatVisualizer, ConnectMeshcatVisualizer

In [ ]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
file_3 = os.path.join(str(Path.home()),
                      "PycharmProjects/quasistatic_simulator/models"
                      "/iiwa7_planar_sphere_collision.sdf")
file_7 = os.path.join(str(Path.home()),
                      "PycharmProjects/quasistatic_simulator/models"
                      "/iiwa7_sphere_collision.sdf")
model_3 = Parser(plant, scene_graph).AddModelFromFile(file_3, "iiwa_7_planar")
model_7 = Parser(plant, scene_graph).AddModelFromFile(file_7)
plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("iiwa_link_0", model_3))
plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("iiwa_link_0", model_7),
                 RigidTransform([0, 1.5, 0]))
plant.Finalize()


frames_to_draw = []
for i in range(8):
    name = f"iiwa_link_{i}"
    body_3 = plant.GetBodyByName(name, model_3)
    body_7 = plant.GetBodyByName(name, model_7)
    frames_to_draw.append(plant.GetBodyFrameIdIfExists(body_3.index()))
    frames_to_draw.append(plant.GetBodyFrameIdIfExists(body_7.index()))


visualizer = ConnectMeshcatVisualizer(builder, scene_graph, frames_to_draw=frames_to_draw)
diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
context_vis = visualizer.GetMyContextFromRoot(context)
context_plant = plant.GetMyContextFromRoot(context)
visualizer.DoPublish(context_vis, [])


In [ ]:
q = np.zeros(7)
q[1] = np.pi / 2
q[2] = np.pi / 2
indices_3_into_7 = [0, 3, 5]

q3 = np.array([np.pi / 6, np.pi / 6, -np.pi / 6])
q[indices_3_into_7] = q3

plant.SetPositions(context_plant, model_7, q)
plant.SetPositions(context_plant, model_3, q[indices_3_into_7])
visualizer.DoPublish(context_vis, [])

In [ ]:
q = np.zeros(7)
q[1] = np.pi / 2
q[2] = np.pi / 2
plant.SetPositions(context_plant, q)
visualizer.DoPublish(context_vis, [])

In [ ]:
body_frames = [plant.GetFrameByName(f"iiwa_link_{i}") for i in range(8)]

In [ ]:
np.set_printoptions(precision=4)
for i, L in enumerate(body_frames):
    X_WL = plant.CalcRelativeTransform(context_plant, plant.world_frame(), L)
    print(i, X_WL.translation(), RollPitchYaw(X_WL.rotation()).vector())
    print('===============================================')